In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
import statsmodels.formula.api as smf

In [37]:
df = pd.read_csv(
    "/data/jerrylee/pjt/BIGFAM.v.2.0/data/supple/UKB.XmXf.bigfam-gcta.weight2.tsv",
    sep='\t'
)
df = df.drop(columns=["weight", "lambda"]).reset_index(drop=True)
df

,pheno,Vx_male_bigfam,lower_male_Vx,upper_male_Vx,Vx_female_bigfam,lower_female_Vx,upper_female_Vx,r,lower_r,upper_r,DCR_bigfam,Vx_male_gcta,lower_Vx_male,upper_Vx_male,Vx_female_gcta,lower_Vx_female,upper_Vx_female,DCR_gcta
0,Hand_grip_strength__left_,0.001706,0.000013,0.004957,0.002045,0.000852,0.003483,1.0,-1.00,1.000,0.834071,0.006059,0.001533,0.010584,0.006665,0.002830,0.010499,0.909057
1,Arm_fat-free_mass__right_,0.008301,0.000077,0.020524,0.005628,0.001430,0.010361,-1.0,-1.00,1.000,1.474962,0.014670,0.009790,0.019549,0.007924,0.003957,0.011891,1.851305
2,Forced_vital_capacity__FVC__Z-score,0.005628,0.000013,0.012107,0.003852,0.001413,0.006939,1.0,-1.00,1.000,1.461019,0.008390,0.002655,0.014125,0.004671,0.000036,0.009307,1.795983
3,Leg_predicted_mass__right_,0.013897,0.004047,0.023310,0.006160,0.002117,0.011632,1.0,-1.00,1.000,2.256149,0.013461,0.008661,0.018260,0.007135,0.003137,0.011132,1.886671
4,Waist_circumference,0.009062,0.002601,0.016144,0.003524,0.000079,0.007412,1.0,-0.43,1.000,2.571106,0.008716,0.004174,0.013258,0.004555,0.000766,0.008344,1.913416
5,Body_fat_percentage,0.006192,0.000001,0.014009,0.003044,0.000033,0.007598,1.0,-1.00,1.000,2.034085,0.010194,0.005505,0.014883,0.005442,0.001511,0.009372,1.873355
6,HDL_cholesterol,0.007531,0.000051,0.017008,0.007337,0.002836,0.013201,1.0,-1.00,1.000,1.026484,0.005962,0.000879,0.011045,0.005346,0.001055,0.009637,1.115130
7,Arm_fat_percentage__left_,0.006884,0.000057,0.012755,0.002913,0.000078,0.006188,1.0,-1.00,1.000,2.363393,0.009404,0.004781,0.014027,0.004946,0.001040,0.008852,1.901325
8,High_light_scatter_reticulocyte_count,0.004676,0.000011,0.009786,0.003237,0.000884,0.005784,-1.0,-1.00,1.000,1.444434,0.010113,0.005363,0.014863,0.007034,0.002908,0.011161,1.437729
9,Platelet_crit,0.010853,0.003584,0.018961,0.006311,0.003135,0.011618,1.0,-1.00,1.000,1.719613,0.010750,0.006206,0.015295,0.004754,0.000941,0.008567,2.261322


In [31]:
df_gs = pd.read_csv(
    "/data/jerrylee/pjt/BIGFAM.v.2.0/data/GS/obj2/prediction-X/X.tsv",
    sep='\t'
)
df_gs = df_gs[df_gs["weight"] == 2].drop(columns=["weight", "lambda"])
df_gs["cohort"] = "GS:SFHS"
df_gs = df_gs[df_gs["lower_Vx"] > 1e-5].reset_index(drop=True)
df_gs = df_gs.rename(columns={"Vx": "BIGFAM", "lower_Vx": "lower_BIGFAM", "upper_Vx": "upper_BIGFAM"}).sort_values(by="BIGFAM", ascending=True).reset_index(drop=True)
df_gs

,pheno,BIGFAM,lower_BIGFAM,upper_BIGFAM,cohort
0,max_arm,0.004098,0.000562,0.007550,GS:SFHS
1,avg_sys,0.004163,0.001830,0.006938,GS:SFHS
2,Total_cholesterol,0.006192,0.001787,0.010085,GS:SFHS
3,avg_dia,0.007466,0.003572,0.010943,GS:SFHS
4,QRS_axis,0.008041,0.006442,0.009520,GS:SFHS
5,P_axis,0.010148,0.008294,0.012491,GS:SFHS
6,PR_interval,0.014417,0.010287,0.019817,GS:SFHS


In [33]:
df_ukb = pd.read_csv(
    "/data/jerrylee/pjt/BIGFAM.v.2.0/data/supple/UKB.X.bigfam-gcta.weight2.tsv",
    sep='\t'
)
df_ukb = df_ukb.drop(columns=["weight"]).sort_values(by="BIGFAM", ascending=True).reset_index(drop=True)
df_ukb["cohort"] = "UKB"
df_ukb

,pheno,GCTA,lower_GCTA,upper_GCTA,BIGFAM,lower_BIGFAM,upper_BIGFAM,cohort
0,LDL_direct,0.002388,0.000205,0.004571,0.004078,0.000174,0.007493,UKB
1,C-reactive_protein,0.002313,0.000151,0.004475,0.004173,0.002534,0.006001,UKB
2,Cholesterol,0.002696,0.000513,0.004878,0.005114,0.000526,0.008962,UKB
3,Apolipoprotein_B,0.002708,0.000525,0.004892,0.005571,0.000820,0.009164,UKB
4,Immature_reticulocyte_fraction,0.003586,0.001388,0.005785,0.007273,0.002277,0.013243,UKB
5,Monocyte_count,0.002645,0.000494,0.004796,0.007828,0.000562,0.013687,UKB
6,SHBG,0.008934,0.006313,0.011555,0.008457,0.001396,0.014424,UKB
7,Waist_circumference,0.004954,0.002788,0.007120,0.011999,0.002736,0.020382,UKB
8,Leg_fat_mass__left_,0.004555,0.002362,0.006747,0.012117,0.002548,0.022514,UKB
9,Leg_fat_percentage__right_,0.005415,0.003194,0.007637,0.012738,0.001651,0.029145,UKB


In [34]:
df_mrg = pd.concat([df_gs, df_ukb]).reset_index(drop=True)
df_mrg

,pheno,BIGFAM,lower_BIGFAM,upper_BIGFAM,cohort,GCTA,lower_GCTA,upper_GCTA
0,max_arm,0.004098,0.000562,0.007550,GS:SFHS,NaN,NaN,NaN
1,avg_sys,0.004163,0.001830,0.006938,GS:SFHS,NaN,NaN,NaN
2,Total_cholesterol,0.006192,0.001787,0.010085,GS:SFHS,NaN,NaN,NaN
3,avg_dia,0.007466,0.003572,0.010943,GS:SFHS,NaN,NaN,NaN
4,QRS_axis,0.008041,0.006442,0.009520,GS:SFHS,NaN,NaN,NaN
5,P_axis,0.010148,0.008294,0.012491,GS:SFHS,NaN,NaN,NaN
6,PR_interval,0.014417,0.010287,0.019817,GS:SFHS,NaN,NaN,NaN
7,LDL_direct,0.004078,0.000174,0.007493,UKB,0.002388,0.000205,0.004571
8,C-reactive_protein,0.004173,0.002534,0.006001,UKB,0.002313,0.000151,0.004475
9,Cholesterol,0.005114,0.000526,0.008962,UKB,0.002696,0.000513,0.004878


In [35]:
df_mrg.to_csv(
    "/data/jerrylee/pjt/BIGFAM.v.2.0/data/supple/obj2.X.with.GCTA.tsv",
    sep='\t',
    index=False
)